## Recorte mediante Shapefile

In [1]:
import os,glob

In [2]:
ruta_archivos = r"E:\Python\Agricultura_Python\Datos\LC09_L2SP_006066_20220626"
os.chdir(ruta_archivos)
os.getcwd()

'E:\\Python\\Agricultura_Python\\Datos\\LC09_L2SP_006066_20220626'

In [3]:
lista = glob.glob("*clip*.tif")
lista

['LC09_L2SP_006066_20220626_RS_B1234567_Clip_extension.tif',
 'LC09_L2SP_006066_20220626_RS_B1234567_Clip_geometria.tif']

In [4]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject

In [41]:
L9_clip = rasterio.open(lista[0])

In [6]:
# Consultar proyeccion
L9_clip.crs

CRS.from_wkt('PROJCS["WGS 84 / UTM zone 18N",GEOGCS["WGS 84",DATUM["World Geodetic System 1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-75],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')

In [7]:
L9_clip.transform

Affine(30.0, 0.0, 504615.0,
       0.0, -30.0, -936975.0)

In [8]:
# Columna de la imagen
L9_clip.width

479

In [9]:
# Fila de la imagen
L9_clip.height

465

In [10]:
# Extension de la imagen
L9_clip.bounds

BoundingBox(left=504615.0, bottom=-950925.0, right=518985.0, top=-936975.0)

In [23]:
from pyproj.crs import CRS

# UTM
new_crs = CRS.from_epsg(32718)

In [24]:
new_transform, width, height = calculate_default_transform(L9_clip.crs, new_crs,
                                                           L9_clip.width, L9_clip.height, 
                                                           *L9_clip.bounds)

In [25]:
new_transform

Affine(29.999999999999563, 0.0, 504614.9999999993,
       0.0, -29.999999999999563, 9063025.0)

In [26]:
param = L9_clip.meta
param

{'driver': 'GTiff',
 'dtype': 'float64',
 'nodata': 0.0,
 'width': 479,
 'height': 465,
 'count': 7,
 'crs': CRS.from_wkt('PROJCS["WGS 84 / UTM zone 18N",GEOGCS["WGS 84",DATUM["World Geodetic System 1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-75],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'),
 'transform': Affine(30.0, 0.0, 504615.0,
        0.0, -30.0, -936975.0)}

In [27]:
# Configurar parametros de mas bandas y tipo dato
param.update(crs = new_crs,
             transform = new_transform)
param

{'driver': 'GTiff',
 'dtype': 'float64',
 'nodata': 0.0,
 'width': 479,
 'height': 465,
 'count': 7,
 'crs': <Derived Projected CRS: EPSG:32718>
 Name: WGS 84 / UTM zone 18S
 Axis Info [cartesian]:
 - E[east]: Easting (metre)
 - N[north]: Northing (metre)
 Area of Use:
 - name: Between 78°W and 72°W, southern hemisphere between 80°S and equator, onshore and offshore. Argentina. Brazil. Chile. Colombia. Ecuador. Peru.
 - bounds: (-78.0, -80.0, -72.0, 0.0)
 Coordinate Operation:
 - name: UTM zone 18S
 - method: Transverse Mercator
 Datum: World Geodetic System 1984 ensemble
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich,
 'transform': Affine(29.999999999999563, 0.0, 504614.9999999993,
        0.0, -29.999999999999563, 9063025.0)}

### Exportar Reproyectado

In [38]:
# Crear nombre de salida
name_RS_PY = os.getcwd() +"\\"+ lista[0][0:25] + "_RS_B1234567_Clip_PY.tif"
name_RS_PY

'E:\\Python\\Agricultura_Python\\Datos\\LC09_L2SP_006066_20220626\\LC09_L2SP_006066_20220626_RS_B1234567_Clip_PY.tif'

In [42]:
import numpy as np

In [43]:
with L9_clip:
    imgdata = np.array([L9_clip.read(i) for i in L9_clip.indexes])

In [44]:
with rasterio.open(name_RS_PY, 'w', **param) as Proj:
    for num, name in enumerate(imgdata, start=1):
        Proj.write_band(num, name)